### Virtual Private Cloud (VPC)
##### Detect IP Address Conflicts

In [ ]:
import ipaddress
import botoplus.botoplus as botoplus

networks = []

accounts = botoplus.accounts()

for account in accounts:

    print(str(account['awsaccount'])+' '+account['awsalias'])
    session = botoplus.defaults(account['awsaccount'])
    ec2 = session.client('ec2')
    regions = ec2.describe_regions()

    for region in regions['Regions']:
        try:
            ec2_region = session.client('ec2', region_name = region['RegionName'])
            paginator = ec2_region.get_paginator('describe_vpcs')
            pages = paginator.paginate()
            for page in pages:
                for vpcs in page['Vpcs']:
                    # Resource Access Manager (RAM) Shared VPC Test
                    if account['awsaccount'] == vpcs['OwnerId']:
                        # Default VPC Test
                        if vpcs['IsDefault'] == True:
                            default = 'Default'
                        else:
                            default = ''
                        # IPv6 Test
                        try:
                            if len(vpcs['Ipv6CidrBlockAssociationSet']) > 0:
                                ipv6 = 'IPv6'
                            else:
                                ipv6 = ''
                        except:
                            ipv6 = ''
                            pass
                        print(' - '+region['RegionName']+' '+vpcs['VpcId']+' '+vpcs['CidrBlock']+' '+default+' '+ipv6)
                        # Convert CIDR to Integer
                        netrange = ipaddress.IPv4Network(vpcs['CidrBlock'])
                        first, last = netrange[0], netrange[-1]
                        firstip = int(ipaddress.IPv4Address(first))
                        lastip = int(ipaddress.IPv4Address(last))
                        networks.append({'vpcid':vpcs['VpcId'],'cidrblock':vpcs['CidrBlock'],'firstip':firstip,'lastip':lastip})
        except:
            print(' - '+region['RegionName']+' DENIED')
            pass

##### Validate IP Address Conflicts

In [ ]:
for network in networks:
    for network2 in networks:
        if network['vpcid'] != network2['vpcid']:
            if network['firstip'] >= network2['firstip'] and network['firstip'] <= network2['lastip']:
                print(' - '+network['vpcid']+' '+network['cidrblock']+' overlaps with '+network2['vpcid']+' '+network2['cidrblock'])
            elif network['lastip'] >= network2['firstip'] and network['lastip'] <= network2['lastip']:
                print(' - '+network['vpcid']+' '+network['cidrblock']+' overlaps with '+network2['vpcid']+' '+network2['cidrblock'])
            elif network['firstip'] <= network2['firstip'] and network['lastip'] >= network2['lastip']:
                print(' - '+network['vpcid']+' '+network['cidrblock']+' overlaps with '+network2['vpcid']+' '+network2['cidrblock'])
            else:
                pass
        else:
            pass